In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices() )

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15091537642247957220
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2000394651
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14745199576930399793
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
%config IPCompleter.greedy=True

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
tf.compat.v1.disable_eager_execution()

In [5]:
n_inputs = 28*28
n_neuronios_camada_1 = 300
n_neuronios_camada_2 = 100
n_neuronios_saida = 10

In [6]:
x = tf.compat.v1.placeholder(shape= (None, n_inputs), dtype= tf.compat.v1.float32, name= 'features')
y = tf.compat.v1.placeholder(shape= (None), dtype= tf.compat.v1.int32, name= 'target')

In [7]:
def camada_Densa(x, nome, n_neuronios, ativacao= None):
    with tf.name_scope(nome):
        n_inputs = int(x.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        # matriz de pesos (muitas vezes chamada de "layer’s kernel"        
        # Ela será um tensor 2D contendo todos os pesos de conexão entre cada entrada e cada neurônio;
        # portanto, sua forma será (n_inputs, n_neurons)
        init = tf.compat.v1.truncated_normal((n_inputs, n_neuronios), stddev= stddev)
        w = tf.Variable(init, name= 'kernel')
        b = tf.Variable( tf.zeros(shape= [n_neuronios]), name= 'bias' )
        z = tf.matmul(x, w) + b
        if ativacao is not None:
            return ativacao(z)
        else:
            return z

In [8]:
with tf.name_scope('rede_neural_profunda'):
    camada_oculta_1 = camada_Densa(x, 'camada_1', n_neuronios_camada_1, tf.nn.relu )
    camada_oculta_2 = camada_Densa(camada_oculta_1, 'camada_2', n_neuronios_camada_2, tf.nn.relu )
    saida = camada_Densa(camada_oculta_2, 'saida', n_neuronios_saida)

In [9]:
with tf.name_scope('loss'):
    x_entropia = tf.nn.sparse_softmax_cross_entropy_with_logits(labels= y, logits= saida)
    loss = tf.reduce_mean(x_entropia, name= 'loss')

In [10]:
lr = 0.01
with tf.name_scope('treino'):
    otimizador = tf.compat.v1.train.GradientDescentOptimizer(lr)
    operacao_treino = otimizador.minimize(loss)

In [12]:
with tf.name_scope('avaliar'):
    correto = tf.math.in_top_k(saida, y, 1, name= 'selecionar_maior_palpite') #tf.cast(y, tf.compat.v1.float32)
    accuracia = tf.reduce_mean(tf.cast(correto, tf.compat.v1.float32) )
iniciar = tf.compat.v1.global_variables_initializer()

TypeError: Value passed to parameter 'targets' has DataType float32 not in list of allowed values: int32, int64